# Functional test with FTDI
- https://www.silabs.com/documents/public/data-sheets/Si5351-B.pdf
- https://www.silabs.com/documents/public/application-notes/AN619.pdf
- https://www.silabs.com/content/usergenerated/asi/cloud/attachments/siliconlabs/en/community/groups/timing/knowledge-base/jcr:content/content/primary/blog/modifying_the_feedba-K8Pv/311668.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline

from clock_generators.si535x.si5351 import Si5351A_B_GT
from utilities.adapters import peripherals

Populating the interactive namespace from numpy and matplotlib


## Debug mode?

In [5]:
cls = Si5351A_B_GT

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators and Modulators

In [6]:
with_hardware_device = False

if with_hardware_device:
    _i2c = peripherals.I2C.get_Ftdi_i2c() 
    
else:
    _i2c =  None  # using None for testing without actual hardware device.

si = cls(_i2c)  

****** Virtual device. Data may not be real ! ******


## Registers values file I/O

In [7]:
si.map.save_to_file('Initial_values.txt')
initial_values = si.map.read_file('Initial_values.txt') 

In [8]:
# clock_builder_values = si.map.read_file('Si5351A-RevB-Registers.txt') 
clock_builder_values = si.map.read_file('Si5351A-RevB-Registers with CenterSS 6ns skew.txt')

In [9]:
# si.write_all_registers(initial_values)

In [10]:
si.map.reset()
si.read_all_registers()
si.map.save_to_file('Read_back_values.txt')
read_back_values = si.map.read_file('Read_back_values.txt')

## Compare Register Value Sets

In [11]:
si.map.compare_values_sets(clock_builder_values, initial_values)

array([[  0.,  nan,   0.,   1.],
       [  1.,  nan,   0.,   1.],
       [  2.,  19.,   0.,   1.],
       [  3.,   0., 248.,   1.],
       [  4.,   0.,  nan,   1.],
       [  7.,   1.,  nan,   1.],
       [  9.,  nan,   0.,   1.],
       [ 15.,   0.,   0.,   0.],
       [ 16.,  15.,  79.,   1.],
       [ 17.,  47.,  79.,   1.],
       [ 18.,  47.,  79.,   1.],
       [ 19., 140., 207.,   1.],
       [ 20., 140., 207.,   1.],
       [ 21., 140., 207.,   1.],
       [ 22., 140., 207.,   1.],
       [ 23., 140., 207.,   1.],
       [ 24.,  nan,   0.,   1.],
       [ 25.,  nan,   0.,   1.],
       [ 26.,   0., 255.,   1.],
       [ 27.,   1., 255.,   1.],
       [ 28.,   0.,   0.,   0.],
       [ 29.,  16.,  16.,   0.],
       [ 30.,   0.,   0.,   0.],
       [ 31.,   0., 240.,   1.],
       [ 32.,   0.,   0.,   0.],
       [ 33.,   0.,   0.,   0.],
       [ 34.,   0., 255.,   1.],
       [ 35.,   1., 255.,   1.],
       [ 36.,   0.,   0.,   0.],
       [ 37.,  16.,  16.,   0.],
       [ 3

In [12]:
si.map.compare_values_sets(initial_values, read_back_values) 

array([[  0.,   0.,   0.,   0.],
       [  1.,   0.,   0.,   0.],
       [  2.,   0.,   0.,   0.],
       [  3., 248.,   0.,   1.],
       [  9.,   0.,   0.,   0.],
       [ 15.,   0.,   0.,   0.],
       [ 16.,  79.,   0.,   1.],
       [ 17.,  79.,   0.,   1.],
       [ 18.,  79.,   0.,   1.],
       [ 19., 207.,   0.,   1.],
       [ 20., 207.,   0.,   1.],
       [ 21., 207.,   0.,   1.],
       [ 22., 207.,   0.,   1.],
       [ 23., 207.,   0.,   1.],
       [ 24.,   0.,   0.,   0.],
       [ 25.,   0.,   0.,   0.],
       [ 26., 255.,   0.,   1.],
       [ 27., 255.,   0.,   1.],
       [ 28.,   0.,   0.,   0.],
       [ 29.,  16.,   0.,   1.],
       [ 30.,   0.,   0.,   0.],
       [ 31., 240.,   0.,   1.],
       [ 32.,   0.,   0.,   0.],
       [ 33.,   0.,   0.,   0.],
       [ 34., 255.,   0.,   1.],
       [ 35., 255.,   0.,   1.],
       [ 36.,   0.,   0.,   0.],
       [ 37.,  16.,   0.,   1.],
       [ 38.,   0.,   0.,   0.],
       [ 39., 240.,   0.,   1.],
       [ 4

## Find Integer Dividers for a Frequency

In [13]:
matchs = si.find_integer_dividers(freq_desired = 90e6, even_only = True, torance_hz = 1) 
matchs

[((36, 10, 1), (25000000, 900000000, 90000000, 90000000))]

In [14]:
matchs = si.find_integer_dividers(freq_desired = 150e6, even_only = True, torance_hz = 1) 
matchs

[((36, 6, 1), (25000000, 900000000, 150000000, 150000000))]

## Find Common Integer Dividers for Frequencies

In [15]:
desired_clock_freqs = (45e6, 30e5, 90e6)
# desired_clock_freqs = (125e6, 48e6, 28.322e6, 74.25e6, 74.25e6, 24.576e6, 22.5792e6, 33.3333e6)
# desired_clock_freqs = (125e6, 48e6, 28.322e6, 74.25e6, 74.25e6, 24.576e6, 22.5792e6, 33.3333e6)
# desired_clock_freqs = (25e6, 25e6)

common_pll_dividers, freqs_pll_dividers, freqs_matches = si.find_integer_pll_dividers_for_clocks(desired_clock_freqs)
common_pll_dividers, freqs_pll_dividers

({36}, [{36}, {24, 30, 36}, {36}])

## Current Configuration

In [16]:
si.init()

In [17]:
si.clocks[0].set_phase(45)

In [18]:
si.current_configuration

,mclk_type,mclk_idx,mclk_freq,pll_idx,pll_divider,pll_freq,multisynth_idx,multisynth_divider,multisynth_freq,multisynth_in_integer_mode,multisynth_divided_by_4,clock_idx,clock_divider,clock_freq,power_downed,enabled,oeb_pin_masked,phase_offset_enabled,phase
0,_Xtal,0,25000000,0,36,900000000,0.0,36.0,25000000.0,False,False,0.0,1.0,25000000.0,False,True,False,True,45.0
1,_Xtal,0,25000000,0,36,900000000,1.0,36.0,25000000.0,True,False,1.0,1.0,25000000.0,False,True,False,False,0.0
2,_Xtal,0,25000000,0,36,900000000,2.0,36.0,25000000.0,True,False,2.0,1.0,25000000.0,False,True,False,False,0.0
3,_Xtal,0,25000000,0,36,900000000,3.0,36.0,25000000.0,True,False,3.0,1.0,25000000.0,True,False,False,False,0.0
4,_Xtal,0,25000000,0,36,900000000,4.0,36.0,25000000.0,True,False,4.0,1.0,25000000.0,True,False,False,False,0.0
5,_Xtal,0,25000000,0,36,900000000,5.0,36.0,25000000.0,True,False,5.0,1.0,25000000.0,True,False,False,False,0.0
6,_Xtal,0,25000000,0,36,900000000,6.0,36.0,25000000.0,True,False,6.0,1.0,25000000.0,True,False,False,False,0.0
7,_Xtal,0,25000000,0,36,900000000,7.0,36.0,25000000.0,True,False,7.0,1.0,25000000.0,True,False,False,False,0.0
8,_Xtal,0,25000000,1,36,900000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Main functions test

In [19]:
for f in dir(cls):
    if not f.startswith('_'):
        print('si.{}()'.format(f))

si.DEBUG_MODE()
si.DEBUG_MODE_PRINT_REGISTER()
si.DEBUG_MODE_SHOW_BUS_DATA()
si.FREQ_REF()
si.HAS_VCXO()
si.I2C_ADDRESS()
si.N_OUTPUT_CLOCKS()
si.OUTPUT_CLOCKS_IN_USE()
si.READ_ONLY_REGISTERS()
si.close()
si.current_configuration()
si.current_frequency()
si.current_phase()
si.do()
si.do_on_devices()
si.enable()
si.enable_output()
si.enable_output_channel()
si.find_integer_dividers()
si.find_integer_pll_dividers_for_clocks()
si.init()
si.is_virtual_device()
si.load_registers()
si.pause()
si.print()
si.read_all_registers()
si.ready()
si.registers_values()
si.reset()
si.reset_plls()
si.restore_clocks_freqs()
si.resume()
si.revision()
si.set_frequency()
si.set_phase()
si.start()
si.status()
si.stop()
si.update()
si.write_all_registers()


In [20]:
si.init()

In [21]:
si.is_virtual_device

True

In [22]:
si.revision

0

In [23]:
# si.set_frequency(2323)

In [24]:
# si.set_phase(55)

In [25]:
si.reset()

In [26]:
# si.current_frequency

In [27]:
# si.current_phase

In [28]:
si.enable()

In [29]:
si.enable_output()

In [30]:
si.enable_output_channel(0)

In [31]:
si.load_registers(initial_values)

In [32]:
si.read_all_registers()

[(0, 'Device_Status', 0),
 (1, 'Interrupt_Status_Sticky', 0),
 (2, 'Interrupt_Status_Mask', 0),
 (3, 'Output_Enable_Control', 0),
 (9, 'OEB_Pin_Enable_Control_Mask', 0),
 (15, 'PLL_Input_Source', 0),
 (16, 'CLK0_Control', 0),
 (17, 'CLK1_Control', 0),
 (18, 'CLK2_Control', 0),
 (19, 'CLK3_Control', 0),
 (20, 'CLK4_Control', 0),
 (21, 'CLK5_Control', 0),
 (22, 'CLK6_Control', 0),
 (23, 'CLK7_Control', 0),
 (24, 'CLK3_0_Disable_State', 0),
 (25, 'CLK7_4_Disable_State', 0),
 (26, 'Multisynth_NA_Parameters_26', 0),
 (27, 'Multisynth_NA_Parameters_27', 0),
 (28, 'Multisynth_NA_Parameters_28', 0),
 (29, 'Multisynth_NA_Parameters_29', 0),
 (30, 'Multisynth_NA_Parameters_30', 0),
 (31, 'Multisynth_NA_Parameters_31', 0),
 (32, 'Multisynth_NA_Parameters_32', 0),
 (33, 'Multisynth_NA_Parameters_33', 0),
 (34, 'Multisynth_NB_Parameters_34', 0),
 (35, 'Multisynth_NB_Parameters_35', 0),
 (36, 'Multisynth_NB_Parameters_36', 0),
 (37, 'Multisynth_NB_Parameters_37', 0),
 (38, 'Multisynth_NB_Parameters_

In [33]:
si.ready

True

In [34]:
si.status.print()


<< Device_Status >>:  ('0x0', '0b0')
[ SYS_INIT ]   :  0
[ LOL_B ]      :  0
[ LOL_A ]      :  0
[ LOS_CLKIN ]  :  0
[ LOS_XTAL ]   :  0
[ Reserved_2 ] :  0
[ REVID ]      :  0


10

In [35]:
si.restore_clocks_freqs()

In [36]:
si.write_all_registers()

In [37]:
si.start()

In [38]:
si.pause()

In [39]:
si.resume()

In [40]:
si.update()

In [41]:
si.stop()

In [42]:
si.close()

In [43]:
si.print()


<< Device_Status >>:  ('0x0', '0b0')
[ SYS_INIT ]   :  0
[ LOL_B ]      :  0
[ LOL_A ]      :  0
[ LOS_CLKIN ]  :  0
[ LOS_XTAL ]   :  0
[ Reserved_2 ] :  0
[ REVID ]      :  0

<< Interrupt_Status_Sticky >>:  ('0x0', '0b0')
[ SYS_INIT_STKY ]  :  0
[ LOL_B_STKY ]     :  0
[ LOL_A_STKY ]     :  0
[ LOS_CLKIN_STKY ] :  0
[ LOS_XTAL_STKY ]  :  0
[ Reserved_0 ]     :  0

<< Interrupt_Status_Mask >>:  ('0x0', '0b0')
[ SYS_INIT_MASK ]   :  0
[ LOL_B_MASK ]      :  0
[ LOL_A_MASK ]      :  0
[ LOS__CLKIN_MASK ] :  0
[ LOS__XTAL_MASK ]  :  0
[ Reserved_0 ]      :  0

<< Output_Enable_Control >>:  ('0xff', '0b11111111')
[ CLK7_OEB ] :  1
[ CLK6_OEB ] :  1
[ CLK5_OEB ] :  1
[ CLK4_OEB ] :  1
[ CLK3_OEB ] :  1
[ CLK2_OEB ] :  1
[ CLK1_OEB ] :  1
[ CLK0_OEB ] :  1

<< OEB_Pin_Enable_Control_Mask >>:  ('0x0', '0b0')
[ OEB_MASK_7 ] :  0
[ OEB_MASK_6 ] :  0
[ OEB_MASK_5 ] :  0
[ OEB_MASK_4 ] :  0
[ OEB_MASK_3 ] :  0
[ OEB_MASK_2 ] :  0
[ OEB_MASK_1 ] :  0
[ OEB_MASK_0 ] :  0

<< PLL_Input_Source >>:

## Spread Spectrum test

In [44]:
si.init()

In [45]:
si.spread_spectrum.enable(True, mode = 'center', ssc_amp = 0.01)

In [46]:
si.spread_spectrum.enable(True, mode = 'down', ssc_amp = 0.01)

In [47]:
si.spread_spectrum.enable(False)

In [48]:
clk = si.clocks[0]